## Before running this notebook, results from the scrapy spider must be available (in the fscrawl directory)
### In order to run the crawler, use the following command (in the fscrawl directory):
```bash
scrapy crawl fs_episodes -O fs-episodes.json -a dlwebvtt=true
```

In [ ]:
import pandas as pd
import plotly.express as px
import pathlib
from datetime import datetime, date

fscrawl_path = pathlib.Path("fscrawl")

plot_output_path = pathlib.Path("plots")
plot_output_path.mkdir(parents=True, exist_ok=True)

write_parquet_files = False
if write_parquet_files:
    processed_output_path = pathlib.Path("processed_dataframes")
    processed_output_path.mkdir(parents=True, exist_ok=True)


## Functions to parse VTT Trancsiption Files
### combining some of the entries for later use with LLMs and RAG

In [ ]:
# combine rows (from top to bottom) until the text length is greater than 500
def combine_rows(df: pd.DataFrame, max_len: int = 500) -> pd.DataFrame:
    combined_rows = []
    current_text = ""
    start = df.iloc[0]["start"]
    end = df.iloc[0]["end"]
    voices = [df.iloc[0]["voice"]]
    for i, row in df.iterrows():
        if len(current_text) + row["text_len"] > max_len:
            duration = end - start
            current_text = current_text.strip()
            combined_rows.append(
                dict(
                    text=current_text,
                    text_len=len(current_text),
                    start=start,
                    end=end,
                    duration=duration.total_seconds(),
                    voices=list(set(voices)),
                )
            )
            current_text = row["text"]
            start = row["start"]
            voices = [row["voice"]]
        else:
            current_text += "\n" + row["text"]
            voices.append(row["voice"])
            end = row["end"]
    combined_rows.append(
        dict(
            text=current_text,
            text_len=len(current_text),
            start=start,
            end=end,
            duration=duration.total_seconds(),
            voices=list(set(voices)),
        )
    )

    df = pd.DataFrame(combined_rows)
    df["n_voices"] = [len(x) for x in df["voices"]]
    return df

# main parsing function
def parse_vtt(
    vttfile: pathlib.Path,
    rec_date: date = None,
    max_len: int = 500,
    combine: bool = True,
) -> pd.DataFrame:
    with open(vttfile) as f:
        alldata = f.read().strip()
        block_strings = alldata.split("\n\n")

    if len(block_strings) <= 2:
        return None

    blocks = []
    for block in block_strings[2:]:
        lines = block.split("\n")
        # if len(lines) <= 2:
        #    return None
        start, end = lines[0].split(" --> ")

        start_time = datetime.strptime(
            start, "%H:%M:%S.%f"
        )  # + timedelta(hours=20) #.time()
        end_time = datetime.strptime(
            end, "%H:%M:%S.%f"
        )  # + timedelta(hours=20)#.time()
        if isinstance(rec_date, date):
            start_time = datetime.combine(rec_date, start_time.time())
            end_time = datetime.combine(rec_date, end_time.time())
        duration = end_time - start_time
        voice, text = lines[1].split(">")
        voice = voice.lstrip("<v ")
        blocks.append(
            dict(
                start=start_time,
                end=end_time,
                duration=duration.total_seconds(),
                voice=voice,
                text=text,
                text_len=len(text),
            )
        )

    if combine:
        return combine_rows(df=pd.DataFrame(blocks), max_len=max_len)
    else:
        return pd.DataFrame(blocks)

### Test: Combined VTT Rows based on text-length (used for embedding - to get a usable context per row)

In [ ]:
df_vtt_combined = parse_vtt(
    vttfile=fscrawl_path / "episode_transcripts/FS136.vtt",
    rec_date=date(year=2014, month=7, day=8),
)
df_vtt_combined.info()

### Test: Not combined - original VTT contents
Sum up the talking durations per speaker

In [ ]:
df_vtt = parse_vtt(
    vttfile=fscrawl_path / "episode_transcripts/FS136.vtt",
    rec_date=date(year=2014, month=7, day=8),
    combine=False,
)
df_vtt.groupby("voice").agg({"duration": "sum"}).sort_values(
    by="duration", ascending=False
)# .sum().values[0]

### Parse all available VTT files (combined) and store them in parquet format
For later use in LLM RAG

In [ ]:
if write_parquet_files:
    vtt_dataframes = []
    for vtt_file in list((fscrawl_path/"episode_transcripts").glob("*.vtt")):
        df_vvt = parse_vtt(vtt_file, max_len=500)
        if df_vvt is not None:
            vtt_dataframes.append(df_vvt)
            df_vvt.to_parquet(processed_output_path / f"df_vtt_{vtt_file.stem}_combined.parquet")
        else:
            print(f"Parsing {vtt_file}: Could not parse")

## Parse crawled episode metadata and combine&check VTT data
- See the printed output for found issues with the VTT transcription files

In [ ]:
df_fs = pd.read_json(fscrawl_path / "fs-episodes.json")

df_fs["year"] = df_fs["date"].dt.year
df_fs["month"] = df_fs["date"].dt.month_name()

# convert guest column to boolean
for col in df_fs.columns:
    if col.startswith("guest:"):
        df_fs[col] = df_fs[col].fillna(False)
        df_fs[col] = df_fs[col].astype(bool)

# sort by date
df_fs = df_fs.sort_values(by="date")
# reset index
df_fs = df_fs.reset_index(drop=True)
df_fs["episode_number"] = df_fs.index + 1

fs_records = []

for i, row in df_fs.iterrows():
    row_record_dict = row.to_dict()

    duration_minutes_total = float(row.duration_minutes)
    vtt_parsing_issue = False
    vtt_speaker_mismatch = False
    vtt_duration_mismatch = False
    if row.webvtt_available:
        vtt_filename = (
            fscrawl_path / f"episode_transcripts/{row.title.split(' ')[0]}.vtt"
        )
        if vtt_filename.exists():
            df_vtt = parse_vtt(vttfile=vtt_filename, combine=False)
            if df_vtt is not None:
                df_vtt_speaker_durations = (
                    df_vtt.groupby("voice")
                    .agg({"duration": "sum"})
                    .sort_values(by="duration", ascending=False)
                )
                vtt_sum_of_speaker_durations_minutes = (
                    df_vtt_speaker_durations.sum().values[0] / 60
                )
                speaker_duration_delta_minutes = (
                    vtt_sum_of_speaker_durations_minutes - duration_minutes_total
                )
                row_record_dict.update(
                    {
                        "vtt_sum_of_speaker_durations_minutes": vtt_sum_of_speaker_durations_minutes,
                    }
                )
                if speaker_duration_delta_minutes > 15:
                    vtt_duration_mismatch = True
                    print(
                        f"! {row.title}: {vtt_filename} - total duration mismatch: VTT is {speaker_duration_delta_minutes:.1f}min longer than description: {vtt_sum_of_speaker_durations_minutes:.1f} > {duration_minutes_total:.1f}"
                    )
                elif speaker_duration_delta_minutes < -15:
                    vtt_duration_mismatch = True
                    print(
                        f"! {row.title}: {vtt_filename} - total duration mismatch: VTT is {-speaker_duration_delta_minutes:.1f}min shorter than description: {vtt_sum_of_speaker_durations_minutes:.1f} < {duration_minutes_total:.1f}"
                    )
                speaker_set_from_description = set(row.guests)
                speaker_set_from_vtt = set(df_vtt_speaker_durations.index)

                if speaker_set_from_description != speaker_set_from_vtt:
                    vtt_speaker_mismatch = True
                    print(
                        f"! {row.title}: {vtt_filename} - speaker mismatch between VTT content and description: {sorted(speaker_set_from_vtt)} != {sorted(speaker_set_from_description)}"
                    )
            else:
                print(
                    f"! {row.title}: {vtt_filename} could not be parsed (the VTT file might be empty)!"
                )
                vtt_parsing_issue = True
        else:
            print(
                f"! {row.title}: {vtt_filename} does not exist (but it should - maybe the crawl-chache was deleted?)!"
            )
            vtt_parsing_issue = True

    row_record_dict.update(
        {
            "vtt_parsing_issue": vtt_parsing_issue,
            "vtt_speaker_mismatch": vtt_speaker_mismatch,
            "vtt_duration_mismatch": vtt_duration_mismatch,
        }
    )
    fs_records.append(row_record_dict)

df_fs = pd.DataFrame.from_records(fs_records)
if write_parquet_files:
    df_fs.to_parquet(processed_output_path / "df_fs_episodes.parquet")
display(df_fs)

## Prepare data for visualizations
  - create a dataframe where each row is a single speaking part of one guest in one episode

In [ ]:
duration_records = []

for i, row in df_fs.iterrows():
    if row.webvtt_available:
        vtt_filename = (
            fscrawl_path / f"episode_transcripts/{row.title.split(' ')[0]}.vtt"
        )
        if vtt_filename.exists():
            df_vtt = parse_vtt(vttfile=vtt_filename, combine=False)
            if df_vtt is not None:
                df_vtt_speaker_durations = (
                    df_vtt.groupby("voice")
                    .agg({"duration": "sum"})
                    .sort_values(by="duration", ascending=False)
                )
                if not any([row.vtt_speaker_mismatch, row.vtt_duration_mismatch]):
                    for speaker, duration in df_vtt_speaker_durations.iterrows():
                        episode_dict = {
                            "episode_number": row.episode_number,
                            "episode_title": row.title,
                            "year": row.year,
                            "month": row.month,
                            "date": row.date,
                            "duration_minutes": duration["duration"] / 60,
                            "speaker": speaker,
                            "vtt_available": row.webvtt_available,
                            "vtt_parsing_issue": row.vtt_parsing_issue,
                            "vtt_speaker_mismatch": row.vtt_speaker_mismatch,
                            "vtt_duration_mismatch": row.vtt_duration_mismatch,
                        }
                        duration_records.append(episode_dict)
                    continue

    duration_minutes_total = float(row.duration_minutes)
    guests = row.guests
    duration_per_guest = duration_minutes_total / len(guests)
    for guest in guests:
        episode_dict = {
            "episode_number": row.episode_number,
            "episode_title": row.title,
            "year": row.year,
            "month": row.month,
            "date": row.date,
            "duration_minutes": duration_per_guest,
            "speaker": guest,
            "vtt_available": row.webvtt_available,
            "vtt_parsing_issue": row.vtt_parsing_issue,
            "vtt_speaker_mismatch": row.vtt_speaker_mismatch,
            "vtt_duration_mismatch": row.vtt_duration_mismatch,
        }
        duration_records.append(episode_dict)

df_speakers_and_durations = pd.DataFrame.from_records(duration_records)
df_speakers_and_durations

## Plots from here on

In [ ]:
for plot_fcn, plot_label in [
    (px.sunburst, "sunburst"),
    (px.treemap, "treemap"),
    (px.icicle, "icicle"),
]:

    fig = plot_fcn(
        df_speakers_and_durations,
        path=[px.Constant("All Years"), "year", "month", "episode_title", "speaker"],
        values="duration_minutes",
        title="FREAKSHOW | Speaker durations in minutes | Year > Month > Episode > Speaker",
        # maxdepth=3,
    )
    # fig.show()
    fig.update_layout(
        margin=dict(l=5, r=5, t=50, b=5),
    )
    fig.write_html(
        plot_output_path/f"{plot_label}_speaker_durations_year>month>episode>speaker.html",
        include_plotlyjs="cdn",
    )

    fig = plot_fcn(
        df_speakers_and_durations,
        path=[px.Constant("All Guests"), "speaker", "year", "month", "episode_title"],
        values="duration_minutes",
        title="FREAKSHOW | Speaker durations in minutes| Speaker > Year > Month > Episode",
        # hovertemplate="<b>%{label}</b><br>%{value} minutes",
        # color="episode_title"
        # maxdepth=2,
    )
    fig.update_layout(
        margin=dict(l=5, r=5, t=50, b=5),
    )
    # fig.show()
    fig.write_html(
        plot_output_path
        / f"{plot_label}_speaker_durations_speaker>year>month>episode.html",
        include_plotlyjs="cdn",
    )

In [ ]:
fig = px.bar(
    df_speakers_and_durations,
    x="episode_number",
    # x="episode_title",
    y="duration_minutes",
    color="speaker",
    hover_data=["episode_title", "date", "vtt_available", "vtt_parsing_issue", "vtt_speaker_mismatch", "vtt_duration_mismatch"],
)
# sort legend by duration
# fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.update_layout(
    title="FREAKSHOW | Speaker durations in minutes | Over Episodes",
    xaxis_title="Episode number",
    yaxis_title="Duration in minutes",
    legend_title="Guests<br>(in order of appearance)",
)
fig.write_html(plot_output_path/"bar_speaker_durations_over_episodes.html", include_plotlyjs="cdn")
fig.show()

In [ ]:
df_cumsum = df_speakers_and_durations.pivot_table(
    index="speaker", 
    columns="date", 
    values="duration_minutes", aggfunc="sum",
).T.cumsum(axis=0)

fig = px.line(
    data_frame=df_cumsum,
    markers=True,
).update_traces(connectgaps=True)
fig.update_layout(
    title="FREAKSHOW | Cumulative speaker durations in minutes | Over Time",
    xaxis_title="Date",
    yaxis_title="Cumulative duration in minutes",
    legend_title="Guests",
)
fig.write_html(plot_output_path/"line_cumsum_speaker_durations_over_time.html", include_plotlyjs="cdn")
fig.show()

In [ ]:
# make year colummn categorical
df_fs["year"] = df_fs["year"].astype(str)

fig = px.scatter(
    data_frame=df_fs,
    x="n_guests",
    y="duration_minutes",
    color="year",
    marginal_x="violin",#"rug",#"violin",#"histogram",
    marginal_y="violin",#"rug",#"violin",#"histogram",
    hover_data=["title", "date"],
    #facet_col="year"
)

fig.update_layout(
    title="FREAKSHOW | Duration in minutes vs. Number of guests",
    xaxis_title="Number of guests",
    yaxis_title="Duration in minutes",
    legend_title="Year",
)
fig.write_html(plot_output_path/"scatter_duration_vs_n_guests.html", include_plotlyjs="cdn")
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=df_fs,
    x="date",
    y="duration_minutes",
    color="webvtt_available",
    color_discrete_map={True: "green", False: "red"},
    size="n_guests",
    hover_data=["title", "guests", "vtt_parsing_issue", "vtt_duration_mismatch", "vtt_speaker_mismatch"],
    symbol="vtt_parsing_issue",
    symbol_sequence=["circle", "x"],
)
fig.update_layout(
    title="FREAKSHOW | Episode Duration and Number of Guests | Over Time",
    xaxis_title="Date",
    yaxis_title="Duration in minutes",
    #legend_title="Transcript available",
)
fig.write_html(plot_output_path/"scatter_duration_vs_date.html", include_plotlyjs="cdn")
fig.show()

In [ ]:
## needs more thinking about...
# fig = px.parallel_categories(
#     data_frame=df_speakers_and_durations,
#     #color="speaker",
#     dimensions=['year',"speaker"],
#     #dimensions=["episode_number",],
#     #labels={"episode_number": "Episode number", "duration_minutes": "Duration in minutes", "speaker": "Speaker"},
#     #color_continuous_scale=px.colors.sequential.Viridis,
# )
# fig.update_layout(
#     title="FREAKSHOW | Speaker durations in minutes | Over Episodes",
#     xaxis_title="Episode number",
#     yaxis_title="Duration in minutes",
# )
# fig.write_html(plot_output_path/"parallel_coordinates_speaker_durations_over_episodes.html", include_plotlyjs="cdn")
# fig.show()

In [ ]:
## I dont like it any more...

# fig = px.density_heatmap(
#     data_frame=df_speakers_and_durations,
#     x="year",
#     y="speaker",
#     z="duration_minutes",
#     color_continuous_scale="reds",

#     #marginal_x="rug",
#     #marginal_y="violin",
#     #color_continuous_scale="Viridis",
#     title="FREAKSHOW | Speaker durations in minutes | Over Years",
# )
# fig.write_html(plot_output_path/"density_heatmap_speaker_durations_over_years.html", include_plotlyjs="cdn")  
# fig.show()


In [ ]:
fig = px.density_heatmap(
    data_frame=df_fs,
    x="year",
    y="month",
    z="duration_minutes",
    color_continuous_scale="reds",
    )
# sort y-axis by month
fig.update_layout(
    title = "FREAKSHOW | Total talking durations in minutes | Over Years and Months",
)
fig.update_yaxes(categoryorder="array", categoryarray=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])
fig.write_html(plot_output_path/"density_heatmap_episode_durations_over_years_and_months.html", include_plotlyjs="cdn")
fig.show()
